In [3]:
import sys

sys.path.append('/orions4-zfs/projects/lins2/Lin_Space/Git_Repos')

from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [4]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean

sys.path.append('/orions4-zfs/projects/lins2/Lin_Space/Git_Repos')

from tf_lab.fundamentals.utils import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category

from tf_lab.autopredictors.scripts import minhyuk_data as minhyuk_loader

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch#,\
                                          
            
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats, paper_pc_completion_experiment_id_best_epoch
                                                  
from geo_tool.fundamentals.cuboid import Cuboid
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


ImportError: cannot import name reset_tf_graph

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [89]:
print minhyuk_loader.test_categories

incomplete_n_samples = 2048
complete_n_samples = 4096

from_manifold = False

category_id = 2
np.random.seed(42)
min_categ_name = minhyuk_loader.test_categories[category_id]
minhyuk_data, accuracies = minhyuk_loader.dataset_of_category(category_id, incomplete_n_samples, complete_n_samples, from_manifold)

['assembly_airplanes', 'assembly_bicycles', 'assembly_chairs', 'coseg_chairs', 'shapenet_tables']
assembly_chairs


In [90]:
l2_norm = np.linalg.norm

def center_axis(points, axis):
    res = points.copy()
    r_max = np.max(res[:, axis])
    r_min = np.min(res[:, axis])
    gap = (r_max + r_min) / 2.0    
    res[:, axis] -= gap
    return res, gap

def z_rotation_by_theta(theta):
    cos_t = np.cos(theta);
    sin_t = np.sin(theta)
    R = np.array([[cos_t, -sin_t, 0], 
                  [sin_t, cos_t, 0],
                  [0, 0, 1]]
                )
    return R

def bounding_box_of_3d_points(points):
    xmin = np.min(points[:, 0])
    xmax = np.max(points[:, 0])
    ymin = np.min(points[:, 1])
    ymax = np.max(points[:, 1])
    zmin = np.min(points[:, 2])
    zmax = np.max(points[:, 2])
    return xmin, ymin, zmin

minh_full_pc, minh_labels, minh_pc_inc = minhyuk_data.full_epoch_data()
n_examples = minhyuk_data.num_examples

new_normalization = True

if category_id == 0:
    shrinks = np.zeros(n_examples)
    shrinkage_multiplier = 3
    for i in xrange(n_examples):
        full_centered, gap = center_axis(minh_full_pc[i], 2)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,2] = minh_pc_inc[i][:,2] - gap

        full_centered, gap = center_axis(minh_full_pc[i], 1)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,1] = minh_pc_inc[i][:,1] - gap

        full_centered, gap = center_axis(minh_full_pc[i], 0)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,0] = minh_pc_inc[i][:,0] - gap

        minh_pc_inc[i] = minh_pc_inc[i].dot(z_rotation_by_theta(0))
        minh_full_pc[i] = minh_full_pc[i].dot(z_rotation_by_theta(0))
        
        xmin, ymin, zmin = bounding_box_of_3d_points(minh_full_pc[i])        
        max_dist = np.max(l2_norm(minh_pc_inc[i], axis=1))
        
        if new_normalization:
            shrinks[i] = np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2
        else:
            shrinks[i] = max_dist * shrinkage_multiplier
       
        print(max_dist * shrinkage_multiplier)
        print(np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2)
        minh_pc_inc[i] /= shrinks[i] 
        minh_full_pc[i] /= shrinks[i]


if category_id == 2:
    shrinks = np.zeros(n_examples)
    shrinkage_multiplier = 2.5
    for i in xrange(n_examples):
        full_centered, gap = center_axis(minh_full_pc[i], 2)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,2] = minh_pc_inc[i][:,2] - gap

        full_centered, gap = center_axis(minh_full_pc[i], 1)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,1] = minh_pc_inc[i][:,1] - gap

        full_centered, gap = center_axis(minh_full_pc[i], 0)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,0] = minh_pc_inc[i][:,0] - gap
        
        #minh_pc_inc[i] = minh_pc_inc[i].dot(z_rotation_by_theta(np.pi / 2 * -1.0 + 0.02))
        #minh_full_pc[i] = minh_full_pc[i].dot(z_rotation_by_theta(np.pi / 2 * -1.0 + 0.02))
        
        if i == 14:
             minh_pc_inc[i] = minh_pc_inc[i].dot(z_rotation_by_theta(np.pi / 2 * -1.0 + 0.02))
             minh_full_pc[i] = minh_full_pc[i].dot(z_rotation_by_theta(np.pi / 2 * -1.0 + 0.02))
        else:
             minh_pc_inc[i] = minh_pc_inc[i].dot(z_rotation_by_theta(np.pi / 2 * -1.0 + 0.02))
             minh_full_pc[i] = minh_full_pc[i].dot(z_rotation_by_theta(np.pi / 2 * -1.0 + 0.02))
        xmin, ymin, zmin = bounding_box_of_3d_points(minh_full_pc[i])
        shrinks[i] = np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2
        
        max_dist = np.max(l2_norm(minh_pc_inc[i], axis=1))
        if new_normalization:
            shrinks[i] = np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2
        else:
            shrinks[i] = max_dist * shrinkage_multiplier
       
        print(max_dist * shrinkage_multiplier)
        print(np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2)
        minh_pc_inc[i] /= shrinks[i] 
        minh_full_pc[i] /= shrinks[i]


if category_id == 3:
    shrinks = np.zeros(n_examples)
    shrinkage_multiplier = 2.2
    for i in xrange(n_examples):
        full_centered, gap = center_axis(minh_full_pc[i], 2)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,2] = minh_pc_inc[i][:,2] - gap

        full_centered, gap = center_axis(minh_full_pc[i], 1)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,1] = minh_pc_inc[i][:,1] - gap

        full_centered, gap = center_axis(minh_full_pc[i], 0)
        minh_full_pc[i] = full_centered
        minh_pc_inc[i][:,0] = minh_pc_inc[i][:,0] - gap

        minh_pc_inc[i] = minh_pc_inc[i].dot(z_rotation_by_theta(np.pi/2*-1.0))
        minh_full_pc[i] = minh_full_pc[i].dot(z_rotation_by_theta(np.pi/2*-1.0))
        
        xmin, ymin, zmin = bounding_box_of_3d_points(minh_full_pc[i])
        shrinks[i] = np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2
        
        max_dist = np.max(l2_norm(minh_pc_inc[i], axis=1))
        
        if new_normalization:
            shrinks[i] = np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2
        else:
            shrinks[i] = max_dist * shrinkage_multiplier
       
        print(max_dist * shrinkage_multiplier)
        print(np.sqrt(xmin ** 2 + ymin ** 2 + zmin ** 2) * 2)
      
        minh_pc_inc[i] /= shrinks[i] 
        minh_full_pc[i] /= shrinks[i]

        
if category_id == 4:
    for i in xrange(n_examples):
        full_centered, gap = center_axis(minh_full_pc[i], 2)
        minh_full_pc[i] = full_centered
        minh_full_pc[i] = minh_full_pc[i].dot(z_rotation_by_theta(np.pi / 2.0))    
        minh_pc_inc[i][:,2] = minh_pc_inc[i][:,2] - gap
        minh_pc_inc[i] = minh_pc_inc[i].dot(z_rotation_by_theta(np.pi / 2.0))        



# Optionally keep input with high default-accuracy:
keep_high_accuracy_only = False
if keep_high_accuracy_only:
    mask = np.array(accuracies) >= 0.99
    minh_full_pc = minh_full_pc[mask]
    minh_labels = minh_labels[mask]
    minh_pc_inc = minh_pc_inc[mask]
    
minhyuk_data_transformed = PointCloudDataSet(minh_full_pc, labels=minh_labels, noise=minh_pc_inc)

print(n_examples)

1.18285515645
1.05112822711
1.22329393128
1.14632559829
1.2470009919
1.00903983047
1.23286915178
1.085087559
1.21509447845
1.03874458761
1.23330601522
1.03695218531
1.23616870656
1.09358149864
1.24362963633
1.12683478608
1.1916149269
1.00584802057
1.24663173284
1.01604527982
1.24102384047
1.04432472769
1.24891585948
1.11061823818
1.24777197708
1.05270217089
1.24509851496
1.04157667193
1.2465465501
1.19017801547
1.2469624841
1.07518908798
1.24663483084
1.04349237906
1.24693326342
1.02418388344
1.24908241359
1.08024790796
1.24895236417
1.06350245576
1.24327781347
1.11193597676
1.24589175354
1.11202979614
1.24365874263
1.05171550363
1.242413179
1.19267982175
1.24946266029
1.04749119848
1.2113462839
1.09983271353
1.24428061858
1.09316172105
1.24248152683
1.08764673155
1.22677199666
1.06242471147
1.24804660655
1.07805344981
1.20670257822
1.0455841097
1.24198675196
1.03738749557
1.23596209682
1.09326731948
1.24800232227
1.04319995548
1.24691878777
1.01175905611
1.24862875537
1.03251860937
1.

In [42]:
GPU = 2

class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

loss = 'chamfer'

experiment_id, best_epoch = paper_pc_completion_experiment_id_best_epoch(class_type, loss)

load_model_conf = True
do_evaluation = True

experiment_name = str(experiment_id) + '_' + class_type + '_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/paper_vanilla_vscan')
train_dir = osp.join(train_dir, experiment_name)

Give me the class type.
chair


In [43]:
if load_model_conf:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    conf.consistent_io = None
    conf.encoder_args = {'dropout_prob': 1.0}
    print conf
else:
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)],
                    'non_linearity': tf.nn.relu
                   }

    encoder_args = {'dropout_prob': 0.5}
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 2,
                learning_rate = 0.0005,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'non_linearity': <function relu at 0x7f404101d758>, 'layer_sizes': [1024, 12288]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'dropout_prob': 1.0}
               experiment_name: 2_chair_2048_4096pts_chamfer
                 gauss_augment: None
                  is_denoising: True
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [4096, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 2
                     train_dir: /orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/2

In [91]:
# Evaluate Minhyuk
testing_data = minhyuk_data_transformed
ae.restore_model(conf.train_dir, best_epoch, verbose=True) 
top_save_dir = osp.join(conf.train_dir, 'output', 'epoch_' + str(best_epoch))
save_dir = osp.join(top_save_dir, 'test_predictions')    

test_recon, test_loss, test_feed, test_ids, test_gt = ae.evaluate(testing_data, conf)

division = True
# Report Accuracy and Coverage of test data.
#n_examples = len(test_recon)
pred_scores = np.zeros((n_examples, 2))
for i in xrange(n_examples):    
    gt = test_gt[i]
    pred = test_recon[i]
    if division:
        pred_scores[i, 0] = accuracy_of_completion(pred , gt , thres=0.02 / shrinks[i], ret_dists=False)
        pred_scores[i, 1] = coverage_of_completion(gt, pred, thres=0.02 / shrinks[i], ret_dists=False)
    else:
        pred_scores[i, 0] = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=False)
        pred_scores[i, 1] = coverage_of_completion(gt, pred, thres=0.02, ret_dists=False)

print 'Test Mean-Median Accuracy:', np.mean(pred_scores[:, 0]), np.median(pred_scores[:, 0])
print 'Test Mean-Median Coverage:', np.mean(pred_scores[:, 1]), np.median(pred_scores[:, 1])

print 'shrinkage_mulitplier ', shrinkage_multiplier

print test_ids
print pred_scores

Model restored in epoch 40.
Test Mean-Median Accuracy: 0.845832824707 0.878784179688
Test Mean-Median Coverage: 0.755695343018 0.76220703125
shrinkage_mulitplier  2.5
['assembly_chairs.4684' 'assembly_chairs.4641' 'assembly_chairs.4176'
 'assembly_chairs.4688' 'assembly_chairs.4135' 'assembly_chairs.4685'
 'assembly_chairs.4136' 'assembly_chairs.4665' 'assembly_chairs.7076'
 'assembly_chairs.7018' 'assembly_chairs.3985' 'assembly_chairs.4067'
 'assembly_chairs.3986' 'assembly_chairs.7062' 'assembly_chairs.3987'
 'assembly_chairs.4190' 'assembly_chairs.7031' 'assembly_chairs.4134'
 'assembly_chairs.3998' 'assembly_chairs.4010' 'assembly_chairs.4132'
 'assembly_chairs.33' 'assembly_chairs.4041' 'assembly_chairs.4679'
 'assembly_chairs.7063' 'assembly_chairs.4663' 'assembly_chairs.7034'
 'assembly_chairs.4051' 'assembly_chairs.4007' 'assembly_chairs.7028'
 'assembly_chairs.4265' 'assembly_chairs.4049' 'assembly_chairs.4175'
 'assembly_chairs.4678' 'assembly_chairs.7030' 'assembly_chairs.4

In [74]:
plt.ioff()
save_dir = osp.join(top_data_dir, 'OUT', 'cmp_minhyuk', loss, min_categ_name)
create_dir(save_dir)

for i in xrange(n_examples):
    gt = test_gt[i]
    pred = test_recon[i]
    feed = test_feed[i]
    model_name = test_ids[i]
    
    ac_meas, ac_col = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=True)
    cov_meas, cov_col = coverage_of_completion(gt, pred, thres=0.02, ret_dists=True)
                    
    ac_col[ac_col <= 0.02] = 0.021
    cov_col[cov_col <= 0.02] = 0.021

    feed_fig = plot_3d_point_cloud_paper(Point_Cloud(points=feed), in_u_sphere=True, show=False);        
    feed_fig.savefig(osp.join(save_dir,  model_name + '.feed.png')); plt.close()

    c = ac_col
    pred_fig = plot_3d_point_cloud_paper(Point_Cloud(points=pred), c=c, in_u_sphere=True, show=False);
    pred_fig.savefig(osp.join(save_dir, model_name + '.prediction.png')); plt.close()

    c = cov_col
    gt_fig = plot_3d_point_cloud_paper(Point_Cloud(points=gt), c=c, in_u_sphere=True, show=False);
    gt_fig.savefig(osp.join(save_dir, model_name + '.gt.png')); plt.close()

NameError: name 'plot_3d_point_cloud_paper' is not defined